## 数据清洗与准备

In [47]:
import pandas as pd
data=pd.read_csv('订单表.csv',encoding='gbk')
# 以订单日期+客户ID为联合索引  每个客户同一日期的订单更有意义
data['订单日期+客户ID']=data['订单日期'].astype('str')+data['客户ID'].astype('str')
orders_series = data.set_index('订单日期+客户ID')['产品名称']
# 排序
orders_series.sort_index(inplace=True)
transactions = []
# 首位ID为11000
temp_index = 10999
# 将每个客户ID对应的产品归置一行
for i, v in orders_series.items():
    if i != temp_index:
        temp_set = set()
        temp_index = i
        temp_set.add(v)
        transactions.append(temp_set)
    else:
        temp_set.add(v)

## 1. efficient_aprior

In [77]:
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.03, min_confidence=0.4)
print('频繁项集：', itemsets)
print('\n')
print('关联规则：', rules)

频繁项集： {1: {('公路自行车',): 8068, ('山地自行车',): 4970, ('车胎和内胎',): 9844, ('头盔',): 6439, ('水壶和水壶架',): 4767, ('骑行服',): 3332, ('手套',): 1430, ('挡泥板',): 2121, ('帽子',): 2190, ('清洁剂',): 908, ('短裤',): 1019, ('旅行自行车',): 2167}, 2: {('头盔', '车胎和内胎'): 2769, ('山地自行车', '水壶和水壶架'): 948, ('山地自行车', '挡泥板'): 898, ('头盔', '水壶和水壶架'): 1203, ('公路自行车', '水壶和水壶架'): 986, ('公路自行车', '头盔'): 1353, ('公路自行车', '车胎和内胎'): 909, ('山地自行车', '车胎和内胎'): 910, ('头盔', '旅行自行车'): 893, ('头盔', '山地自行车'): 934}}


关联规则： [{头盔} -> {车胎和内胎}, {挡泥板} -> {山地自行车}, {旅行自行车} -> {头盔}]


## 2. mlxtend

In [27]:
#  传入模型的数据需要满足特定的格式，转换为bool值
from mlxtend.preprocessing import TransactionEncoder#
te = TransactionEncoder()
df_tf = te.fit_transform(transactions)
# 将 True、False 转换为 0、1 
df_01 = df_tf.astype('int')
encoded_df= pd.DataFrame(df_01,columns=te.columns_)

In [69]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(encoded_df, min_support=0.03, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support',ascending=False)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.3)
rules= rules.sort_values(by='lift',ascending=False)
print("频繁项集：", frequent_itemsets)
print("关联规则：", rules)

频繁项集：      support         itemsets
10  0.356434          (车胎和内胎)
0   0.292128          (公路自行车)
1   0.233145             (头盔)
2   0.179955          (山地自行车)
7   0.172605         (水壶和水壶架)
11  0.120646            (骑行服)
18  0.100261      (车胎和内胎, 头盔)
3   0.079296             (帽子)
6   0.078463          (旅行自行车)
5   0.076798            (挡泥板)
4   0.051778             (手套)
12  0.048990      (公路自行车, 头盔)
17  0.043559     (水壶和水壶架, 头盔)
9   0.036896             (短裤)
13  0.035701  (公路自行车, 水壶和水壶架)
20  0.034325  (水壶和水壶架, 山地自行车)
15  0.033819      (山地自行车, 头盔)
21  0.032950   (山地自行车, 车胎和内胎)
14  0.032913   (公路自行车, 车胎和内胎)
8   0.032877            (清洁剂)
19  0.032515     (山地自行车, 挡泥板)
16  0.032334      (头盔, 旅行自行车)
关联规则：   antecedents consequents  antecedent support  consequent support   support  \
0     (山地自行车)       (挡泥板)            0.179955            0.076798  0.032515   
1       (挡泥板)     (山地自行车)            0.076798            0.179955  0.032515   
2        (头盔)     (旅行自行车)            0.233145            0.07